In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import random

import tensorboard
# Set random seed for reproducibility
tf.random.set_seed(777)

# Load MNIST dataset
mnist = tfds.load(name='mnist', split='train', as_supervised=True, data_dir="MNIST_data/")

# Define constants
nb_classes = 10
input_shape = (28, 28, 1)  # MNIST images are 28x28 pixels

# Create dataset
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalize images to [0, 1]
    image = tf.reshape(image, [784])  # Flatten the image
    return image, tf.one_hot(label, nb_classes)

train_dataset = mnist.map(preprocess).batch(100)

# Define a function to create and compile the model
def create_model(learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(256, activation='relu', input_shape=(784,)),
        tf.keras.layers.Dense(nb_classes, activation='softmax')
    ])
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Define the learning rate and optimizer



# Compile model with the optimizer and learning rate
model=create_model(0.01)

# Train model
model.fit(train_dataset, epochs=15)

# Test the model
test_mnist = tfds.load(name='mnist', split='test', as_supervised=True, data_dir="MNIST_data/")
test_dataset = test_mnist.map(preprocess).batch(100)

test_loss, test_accuracy = model.evaluate(test_dataset)
print("Test Accuracy: {:.4f}".format(test_accuracy))

# Predict and display an image
for images, labels in test_dataset.take(1):  # Get a batch of images and labels
    r = random.randint(0, images.shape[0] - 1)  # Select a random image from the batch
    plt.imshow(images[r].numpy().reshape(28, 28), cmap='Greys', interpolation='nearest')
    plt.title(f"Label: {tf.argmax(labels[r]).numpy()}, Prediction: {tf.argmax(model(images)[r]).numpy()}")
    plt.show()

In [ ]:
import os
# Create a directory for TensorBoard logs
log_dir = "logs/"
os.makedirs(log_dir, exist_ok=True)

# Define different learning rates
learning_rates = [0.01, 0.1,]

# Train models with different learning rates and log metrics
for lr in learning_rates:
    model = create_model(learning_rate=lr)
    
    # Create a TensorBoard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(log_dir, f"lr_{lr}"),
        histogram_freq=1
    )
    
    # Train the model
    history = model.fit(train_dataset, epochs=20, validation_data=test_dataset, callbacks=[tensorboard_callback])

# Launch TensorBoard in the terminal (or follow instructions if using Jupyter Notebook)
# %tensorboard --logdir logs/

print("Training finished. Check TensorBoard for visualization.")

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/